# Generate word2vec model - include all new topic words

In [1]:
from gensim.models import Word2Vec, KeyedVectors
import gensim
import time
import numpy as np
from tqdm import tqdm
from pprint import pprint
import pickle
import spacy
import re
import operator
import spacy_udpipe

In [2]:
nlp = spacy.load('es_core_news_sm')

In [3]:
# set variables 
LANG = 'es'
MAX_WORDS = 500000  # maximum processed words to small model 

MODEL_DIR = '/mnt/data/data/models-fasttext/'
TOPICS_DIR = '/home/dzon/kajo/topics/topic_lists-based_on_en-US/'

model_filename = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2.vec.bin'

excluded_model_name = MODEL_DIR + 'cc.'+ LANG +'.300-lemma_500k_min2_excluded.vec.bin'

model_orig_filename = MODEL_DIR + 'cc.'+ LANG +'.300.vec.bin'
model_orig_filename_vec = MODEL_DIR + 'cc.'+ LANG +'.300.vec.gz'

pickled_words_lemmas = MODEL_DIR + LANG + '_ft_all_words_lemmas.pickle'

lemma_list_file = TOPICS_DIR + 'topics_lemma_'+ LANG +'.lst'
nolemma_list_file = TOPICS_DIR + 'topics_'+ LANG +'.lst'

excluded_filename = TOPICS_DIR + 'topics_' + LANG + '-not_in_model.lst'

lemma_list_file_excluded = TOPICS_DIR + 'topics_lemma_excluded_' + LANG + '.lst'
nolemma_list_file_excluded = TOPICS_DIR + 'topics_excluded_' + LANG + '.lst'


In [4]:
# load the big model
model_big = KeyedVectors.load_word2vec_format(model_orig_filename, binary=True)

In [5]:
#load pickled lemma list
with open(pickled_words_lemmas, 'rb') as f:
    all_words_lemmas = pickle.load(f)
pprint(all_words_lemmas[:50])

[('de', 'de'),
 ('la', 'lo'),
 ('y', 'y'),
 ('en', 'en'),
 ('que', 'que'),
 ('el', 'el'),
 ('a', 'a'),
 ('los', 'lo'),
 ('del', 'del'),
 ('un', 'uno'),
 ('se', 'se'),
 ('con', 'con'),
 ('por', 'por'),
 ('las', 'los'),
 ('para', 'parir'),
 ('una', 'uno'),
 ('es', 'ser'),
 ('no', 'no'),
 ('su', 'su'),
 ('al', 'al'),
 ('como', 'comer'),
 ('lo', 'el'),
 ('más', 'más'),
 ('El', 'el'),
 ('o', 'o'),
 ('La', 'la'),
 ('me', 'me'),
 ('En', 'en'),
 ('sus', 'su'),
 ('este', 'este'),
 ('pero', 'pero'),
 ('ha', 'haber'),
 ('esta', 'este'),
 ('_', '_'),
 ('si', 'si'),
 ('sobre', 'sobrar'),
 ('fue', 'ser'),
 ('son', 'ser'),
 ('le', 'le'),
 ('muy', 'muy'),
 ('ser', 'ser'),
 ('ya', 'ya'),
 ('tu', 'tu'),
 ('todo', 'todo'),
 ('entre', 'entrar'),
 ('te', 'te'),
 ('mi', 'mi'),
 ('Los', 'los'),
 ('sin', 'sin'),
 ('también', 'también')]


In [6]:
# generate helpful dictionaries/lists
big_lemmas_set = set([l for (_,l) in all_words_lemmas])
print(len(big_lemmas_set))

big_words_set = set([w for (w,_) in all_words_lemmas])
print(len(big_words_set))

lemmas = {}
words_lower_orig = {}
big_lemma_word_dic = {}
big_word_lemma_dic = {}
for (w,l) in all_words_lemmas:
    big_word_lemma_dic[w] = l
    
    #dictionary of lowercase word -> all original words
    if w.lower() in words_lower_orig:
        words_lower_orig[w.lower()].append(w)
    else:
        words_lower_orig[w.lower()] = [w]
        
    if l in big_lemma_word_dic:
        big_lemma_word_dic[l].append(w)
        lemmas[l] += 1
    else:
        big_lemma_word_dic[l] = [w]
        lemmas[l] = 1
        
# load all the words from big model
words = model_big.wv.index2word
words_lower = [w.lower() for w in words]

#words_lemma = model_lemma_big.index2word       

1099975
1528979


/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [7]:
# sort the lemmas from most frequent
sorted_lemmas = sorted(lemmas.items(), key=operator.itemgetter(1), reverse=True)
# trying first words
sorted_lemmas[:20]

[('ser', 58),
 ('estar', 58),
 ('hacer', 58),
 ('tener', 57),
 ('decir', 57),
 ('haber', 56),
 ('encontrar', 56),
 ('llegar', 56),
 ('poder', 55),
 ('creer', 55),
 ('poner', 55),
 ('querer', 55),
 ('llevar', 55),
 ('venir', 55),
 ('pedir', 55),
 ('ver', 54),
 ('saber', 54),
 ('dejar', 54),
 ('hablar', 54),
 ('salir', 54)]

In [8]:
# create vectors for the small models - average vectors for lemmas
# process only first MAX_WORDS in the big model
tmp_words = [None] * len(model_big.wv.index2word)
tmp_vecs = [None] * len(model_big.wv.index2word) 
for i,(word,lemma) in enumerate(all_words_lemmas[:MAX_WORDS]):
    lemma_words_list = big_lemma_word_dic[lemma]
    # add only lemmas made from at least 2 words
    if len(lemma_words_list)<2:
        continue
    tmp_vectors = []
    tmp_indexes = []
    error_with_word = False
    for word2 in lemma_words_list:
        try:
            tmp_vectors.append(model_big[word2])  
            tmp_indexes.append(model_big.vocab[word2].index)
        except:
            print(i, "error with ", word, " lemma ", lemma, "words", lemma_words_list)
            error_with_word = True
            continue
    if not error_with_word:
        try:
            tmp_words[min(tmp_indexes)] = lemma
            tmp_vecs[min(tmp_indexes)] = list(np.average(tmp_vectors, axis=0))
        except:
            print(i, "error with lemma ", lemma, " - words", lemma_words_list, " indexes: ", tmp_indexes)
            # note - we are processing first MAX_WORDS from the model after the filter out of the "non-words"
            # so some of the last words in that range may refer to words on bigger index than MAX_WORDS

# filter the lists (some positions remained None)
final_words = [i for i in tmp_words if i] 
final_vecs = [i for i in tmp_vecs if i != None] 

print(len(final_vecs), len(final_words))
len(final_vecs[0])  

del tmp_vecs
del tmp_words

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


165809 165809


In [9]:
# create new gensim word2vec model with the vectors:
new_vector_space = gensim.models.keyedvectors.Word2VecKeyedVectors(300)
new_vector_space.add(final_words,final_vecs)

In [10]:
# test the model
pprint(new_vector_space.most_similar('padre'))
pprint(new_vector_space.most_similar_cosmul(positive=['mujer', 'rey'], negative=['hombre']))

[('hijo', 0.8011560440063477),
 ('papá', 0.6809394359588623),
 ('madre', 0.6600850224494934),
 ('hermano', 0.6371493339538574),
 ('amadísimo', 0.6195145845413208),
 ('sobrino', 0.6174801588058472),
 ('hijito', 0.6055609583854675),
 ('esposo', 0.6052435040473938),
 ('progenitor', 0.5952569246292114),
 ('primogénito', 0.5942906141281128)]
[('reina', 0.9104922413825989),
 ('princesa', 0.8632273077964783),
 ('reyna', 0.8456254601478577),
 ('emperatriz', 0.8270469903945923),
 ('majestad', 0.8195383548736572),
 ('reinas', 0.808714747428894),
 ('realeza', 0.8063253164291382),
 ('coronada', 0.7932454347610474),
 ('coronación', 0.7908713221549988),
 ('infantas', 0.7907785773277283)]


In [11]:
# save the model
new_vector_space.save_word2vec_format(model_filename, binary=True)

In [12]:
#model = KeyedVectors.load_word2vec_format(model_filename, binary=True)
model = new_vector_space
words_model = model.wv.index2word

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# load topic listst
with open(nolemma_list_file) as fp:
    topics_nolemma = fp.readlines() 
print(len(topics_nolemma))
    
# load the lemma list
# with open(lemma_list_file) as fp:
#    topics = fp.readlines()
# print(len(topics))

topics = []
# generate the lemma list
for topic in tqdm(topics_nolemma):
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []
        
    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    if len(lemmas) == 0:
        print('lemmas = 0 ', topic)
    topics.append("_".join(lemmas))
    
print(len(topics))    
    
#write the lemma topic list
with open(lemma_list_file, 'wt', encoding='utf8') as f:
    for topic_lemma in topics:
        f.write(topic_lemma + "\n")    

  0%|          | 13/5308 [00:00<00:41, 126.42it/s]

5308


 13%|█▎        | 700/5308 [00:03<00:21, 215.18it/s]

lemmas = 0  empleo 


 21%|██▏       | 1138/5308 [00:05<00:19, 212.46it/s]

...

lemmas = 0  fuera de lugar 


 88%|████████▊ | 4693/5308 [00:23<00:02, 207.10it/s]

lemmas = 0  embargo 


100%|██████████| 5308/5308 [00:26<00:00, 203.25it/s]

5308


In [14]:
print(len(topics),len(topics_nolemma))  

5308 5308


In [15]:
#try if the topic-lemma words exist in the models  

words_to_manual_add = {}

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic = topic.strip()
    topic_words = topic.split('_')
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    for word, word_nolemma in zip(topic_words, nolemmas):
        #print(word)
        found = 0
        if word in words_model:
            found = 1
            #print("l1")
        elif word in big_lemmas_set:
            found = 1
            print("l2", word)
        elif word in words_lower:
            found = 1
            print("l3", word)
        # nolemma words are in PL case probably not needed and all lemmas exist in lemma list
        elif word_nolemma in words_model:
            found = 1
            print("w1", word)
        elif word_nolemma in big_lemmas_set:
            found = 1
            print("w2", word)
        elif word_nolemma in words_lower:
            found = 1
            print("w3", word)

        if found == 0:
            print(i, 'needs to be added manually: ', word, " - ", word_nolemma, " - ", topic_nolemma.strip())
            if word not in words_to_manual_add:
                words_to_manual_add[word] = topic_nolemma.strip()
                    
print(len(words_to_manual_add))          

l2 geofisico
l2 microclimático
650 needs to be added manually:  reglementación  -  reglementación  -  reglementación del transporte
!!!!! error - not equal length:  [] ['']
l2 ostricultura
732 needs to be added manually:  hidrálica  -  hidrálica  -  hidrálica agricola
873 needs to be added manually:  circulatión  -  circulatión  -  reglamentación de la circulatión
l2 ripícola
l2 excrementar
972 needs to be added manually:  demesnial  -  demesnial  -  agua demesnial
l2 geomórfico
l2 aterrazado
!!!!! error - not equal length:  [] ['']
1181 needs to be added manually:  demesnial  -  demesnial  -  agua no demesnial
l2 manatial
l2 sedimientos
1382 needs to be added manually:  termoselectivo  -  termoselectivo  -  proceso termoselectivo
l2 nidificantes
l2 extración
l2 squatter
!!!!! error - not equal length:  [] ['']
l3 co2
l2 béntico
!!!!! error - not equal length:  [] ['']
l2 aox
1951 needs to be added manually:  gelisuelo  -  gelisuelo  -  ecosistema de gelisuelo
1961 needs to be added ma

5271 needs to be added manually:  micetes  -  micetes  -  micetes
l2 gallinacea
l2 geotecnología
5276 needs to be added manually:  haloforme  -  haloforme  -  haloforme
5280 needs to be added manually:  lisimetría  -  lisimetría  -  lisimetría
5281 needs to be added manually:  metainformation  -  metainformation  -  metainformation
5282 needs to be added manually:  microecosystem  -  microecosystem  -  microecosystem
5284 needs to be added manually:  microcontaminante  -  microcontaminante  -  microcontaminante
l2 sobrexplotación
5287 needs to be added manually:  superfertilización  -  superfertilización  -  superfertilización
5288 needs to be added manually:  peroxiacetilo  -  peroxiacetilo  -  nitrato de peroxiacetilo
l2 fitomasa
l2 dibenzofurano
5292 needs to be added manually:  policlorinado  -  policlorinado  -  dibenzofurano policlorinado
l2 autoecología
5294 needs to be added manually:  proboscideo  -  proboscideo  -  proboscideo
5301 needs to be added manually:  traqueofita  - 

In [16]:
# for manual correcting of words
pprint(words_to_manual_add)

{'absorpción': 'absorpción',
 'atracina': 'atracina',
 'biofiltrado': 'biofiltrado',
 'bioluminescencia': 'bioluminescencia',
 'bocaje': 'bocaje',
 'bradiseísmo': 'bradiseísmo',
 'cetrifugado': 'cetrifugado proceso',
 'chamizado': 'cultivo sobre chamizado',
 'circulatión': 'reglamentación de la circulatión',
 'cloroetilenos': 'cloroetilenos',
 'clorofluorohidrocarburos': 'clorofluorohidrocarburos',
 'coloideo': 'coloideo coloidal',
 'colorimetriea': 'colorimetriea',
 'compostificación': 'compostificación por el productor',
 'contaminacíon': 'contaminacíon radiactiva',
 'crisofitas': 'crisofitas',
 'demesnial': 'agua demesnial',
 'dendrometría': 'dendrometría',
 'dermapterans': 'dermapterans',
 'desasorción': 'desasorción',
 'ecocatástrofe': 'ecocatástrofe',
 'ecomarketing': 'ecomarketing',
 'ecopediatría': 'ecopediatría',
 'ecozone': 'ecozone',
 'emisiónes': 'emisiónes de gases de efecto invernadero',
 'enselvamiento': 'enselvamiento',
 'exación': 'exación',
 'explotacíon': 'régimen de

In [17]:
# the lemmas not in model were manually checked if they exist in other models in some variation
word = "φυτοκομείο"
print(word)
try:
    model[word]
    print('model')
    pprint(model.most_similar(word))
except:
    pass

if word in big_lemmas_set:
    print('model_lemma_big', big_lemma_word_dic[word])

try:
    model_big[word]
    print('model big')
    pprint(model_big.most_similar(word))
except:
    pass

# if the original word with uppercase
if word in words_lower_orig:
    print('model big LOWER', words_lower_orig[word])
    pprint(model_big.most_similar(words_lower_orig[word]))
    

φυτοκομείο


In [18]:
# for cases the lemmas aren't correct or don't exist in the dic
# manually made dictionary of "lemma" : "word which exists in some model" or "word1;word2" for the average
# e.g. 'mikroekosystém': 'mikro;ekosystém', 'bahný': 'bahno'
new_words_manually_checked = {}
    
len(new_words_manually_checked)

0

In [19]:
# function which finds the word in all the available models and returns it's vector (average vector)
def get_vector(word):
    # word is in small model
    try:
        return model[word]
    except:
        pass
    
    # word is in lemmas from big model - return average if the lemma was result of more words
    if word in big_lemmas_set:
        lemma_words_list = big_lemma_word_dic[word]
        tmp_vectors = []
        for word2 in lemma_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

    try:
        return model_big[word]
    except:
        pass
    
    # if the original word with uppercase
    if word in words_lower_orig:
        orig_words_list = words_lower_orig[word]
        tmp_vectors = []
        for word2 in orig_words_list:
            tmp_vectors.append(model_big[word2])  
        return np.average(tmp_vectors, axis=0)

        #return model_big[words_lower_orig[word]]
    
    return []

In [20]:
# check if the word vectors are the same
#get_vector('Antarktyki') == get_vector('antarktyki')
get_vector('διβενζοφουράνιο')

[]

In [21]:
# ading words from manually checked list
new_vectors = []
new_words = []
for i, word in enumerate(new_words_manually_checked.keys()):
    
    tmp_vectors = []
    print(i, word)
    for tmp_word in new_words_manually_checked[word].split(';'):
        vector = get_vector(tmp_word)
        if vector != []:
            tmp_vectors.append(vector)
        else:
            print("XXX - some ERROR, word doesn't have vector, cannot add: ", tmp_word)
    if word not in new_words:
        new_vectors.append(np.average(tmp_vectors, axis=0))
        new_words.append(word)
print(len(new_vectors),len(new_words))        

0 0


In [22]:
# add new words to the model
if len(new_words) > 0:
    model.add(new_words,new_vectors)

In [23]:
print(len(topics),len(topics_nolemma))  

5308 5308


In [24]:
# 2nd run - checking if there are still unavailable words in topics
excluded_topics = []
#try if the topic-lemma words exist in the model   

for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
    # process the nolemma topic so we can see which word was lemmatized to which lemma
    topic_nolemma_orig = topic_nolemma.strip()
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text)  
    
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        #mostly because the lemma topic is empty ""
        excluded_topics.append(topic_nolemma_orig)
        continue
        
    found = 1
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        #print(word)
       
        vector = get_vector(word)
        if vector == []:
            print('nonexistend word: ', word, " - ", topic_nolemma)
            vector2 = get_vector(word_nolemma)
            if vector2 == []:
                found = 0
                print('nonexistend word: ', word_nolemma, " - ", topic_nolemma)
            else:
                print("Exist nolemma")

    if found == 0:
        print('!!! topic needs to be excluded: ', topic_nolemma_orig)
        if topic_nolemma.strip() not in excluded_topics:
            excluded_topics.append(topic_nolemma_orig)
                    
print(len(excluded_topics)) 

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


nonexistend word:  reglementación  -  reglementación del transporte 
nonexistend word:  reglementación  -  reglementación del transporte 
!!! topic needs to be excluded:  reglementación del transporte
!!!!! error - not equal length:  [] ['']
nonexistend word:  hidrálica  -  hidrálica agricola 
nonexistend word:  hidrálica  -  hidrálica agricola 
!!! topic needs to be excluded:  hidrálica agricola
nonexistend word:  circulatión  -  reglamentación de la circulatión 
nonexistend word:  circulatión  -  reglamentación de la circulatión 
!!! topic needs to be excluded:  reglamentación de la circulatión
nonexistend word:  demesnial  -  agua demesnial 
nonexistend word:  demesnial  -  agua demesnial 
!!! topic needs to be excluded:  agua demesnial
!!!!! error - not equal length:  [] ['']
nonexistend word:  demesnial  -  agua no demesnial 
nonexistend word:  demesnial  -  agua no demesnial 
!!! topic needs to be excluded:  agua no demesnial
nonexistend word:  termoselectivo  -  proceso termosel

nonexistend word:  terephtalate  -  polietileno terephtalate 
nonexistend word:  terephtalate  -  polietileno terephtalate 
!!! topic needs to be excluded:  polietileno terephtalate
nonexistend word:  terfenilo  -  terfenilos policlorados 
nonexistend word:  terfenilos  -  terfenilos policlorados 
!!! topic needs to be excluded:  terfenilos policlorados
nonexistend word:  bioluminescencia  -  bioluminescencia 
nonexistend word:  bioluminescencia  -  bioluminescencia 
!!! topic needs to be excluded:  bioluminescencia
nonexistend word:  xenobiótica  -  sustancia xenobiótica 
nonexistend word:  xenobiótica  -  sustancia xenobiótica 
!!! topic needs to be excluded:  sustancia xenobiótica
nonexistend word:  clorofluorohidrocarburos  -  clorofluorohidrocarburos 
nonexistend word:  clorofluorohidrocarburos  -  clorofluorohidrocarburos 
!!! topic needs to be excluded:  clorofluorohidrocarburos
nonexistend word:  restituible  -  envase no restituible 
nonexistend word:  restituible  -  envase n

In [25]:
# save topics containing words which are not present neither model - to be excluded from the topic list
print('excluded topics: ', len(excluded_topics))
pprint(excluded_topics)
with open(excluded_filename, 'wt', encoding='utf8') as f:
    for topic in excluded_topics:
        f.write(topic + "\n")

excluded topics:  92
['reglementación del transporte',
 'empleo',
 'hidrálica agricola',
 'reglamentación de la circulatión',
 'agua demesnial',
 'trabajo',
 'agua no demesnial',
 'proceso termoselectivo',
 'tiempo',
 'estado del tiempo',
 'ecosistema de gelisuelo',
 'libre circulatión de capitales',
 'TRGS <D>',
 'lixivización',
 'sustencias inorgánicas',
 'macrofito',
 'absorpción',
 'desasorción',
 'oligosustancias y residuos',
 'ombrices de tierra',
 'compostificación por el productor',
 'cubierta humidificadora',
 'compostificación',
 'habla',
 'compuestos de organocloro',
 'compuestos órganohalogenados',
 'compuestos organonitrogenados',
 'contaminacíon radiactiva',
 'actividad socioeducational',
 'coloideo, coloidal',
 'régimen de explotacíon',
 'periphyton',
 'lucha contra la contaminacíon',
 'lugar de trabajo',
 'paraquímica',
 'retroadaptación',
 'estado',
 'cetrifugado (proceso)',
 'bocaje',
 'compuestos órganofosforados',
 'fuera de lugar',
 'atracina',
 'renaturación',
 'm

In [26]:
# create new lists which don't containt topics with non-existent words

# load list with excluded topics
with open(excluded_filename, 'rt', encoding='utf8') as fp:
    tmp_topics = fp.readlines()
excluded_set = set()
for topic in tmp_topics:
    excluded_set.add(topic.strip())

with open(nolemma_list_file, 'rt', encoding='utf8') as fp:
        topics_file = fp.readlines()

topics_final = []
topics_lemma = []

excluded = 0
for topic in topics_file:
    topic_orig = topic.strip()
    if topic_orig in excluded_set:
        print("=== EXCLUDE ", topic_orig)
        excluded += 1
        continue
    topic = re.sub('\W',' ',topic)
    topic = re.sub('\ +',' ',topic)
    doc = nlp(topic.strip().lower())
    lemmas = []

    for token in doc:
        if token.is_stop:
            continue
        lemmas.append(token.lemma_)    
    topics_lemma.append("_".join(lemmas))
    topics_final.append(topic_orig)

    print(topic_orig, ' --- ', "_".join(lemmas))

print('Excluded: ', excluded)
print('Final topics: ', len(topics_lemma), len(topics_final))
    
with open(lemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_lemma in topics_lemma:
        f.write(topic_lemma + "\n")
with open(nolemma_list_file_excluded, 'wt', encoding='utf8') as f:
    for topic_final in topics_final:
        f.write(topic_final + "\n")  

desarrollo local  ---  desarrollar_local
desarrollo rural  ---  desarrollar_rural
gestión de recursos de agua  ---  gestión_recurso_aguar
nivel de enseñanza  ---  nivel_enseñanza
enseñanza general  ---  enseñanza
ribera  ---  ribera
derecho comunitario  ---  derecho_comunitario
patrimonio cultural  ---  patrimonio_cultural
suelo agrícola  ---  soler_agrícola
paisaje agrícola  ---  paisaje_agrícola
producción agrícola  ---  producción_agrícola
desarrollo económico  ---  desarrollar_económico
cadena alimentaria  ---  cadena_alimentario
industria de la alimentación  ---  industriar_alimentación
valor nutritivo  ---  nutritivo
calentamiento de la tierra  ---  calentamiento_tierra
efecto invernadero  ---  efecto_invernadero
desarrollo industrial  ---  desarrollar_industrial
mercado de trabajo  ---  mercar
acceso a la tierra  ---  acceso_a_tierra
consolidación del suelo  ---  consolidación_soler
ordenación del territorio  ---  ordenación_territorio
arquitectura del paisaje  ---  arquitectura

legislación agrícola  ---  legislación_agrícola
sanidad  ---  sanidad
extensión de asentamientos  ---  extensión_asentamiento
recursos  ---  recurso
agua de drenaje  ---  aguar_drenaje
estructura regional  ---  estructurar_regional
estructuras agrarias  ---  estructurar_agrario
gente desprotegida  ---  gente_desproteger
infraestructura de tráfico  ---  infraestructura_tráfico
sistema de seguridad  ---  sistema_seguridad
infraestructuras  ---  infraestructura
descontaminación de suelo  ---  descontaminación_soler
pastos  ---  pasto
crecimiento económico  ---  crecimiento_económico
degradación de agua dulce  ---  degradación_aguar_dulce
alimentos  ---  alimento
acidificación de suelo  ---  acidificación_soler
patrimonio biológico  ---  patrimonio_biológico
acto comunitario  ---  acto_comunitario
edificio agrícola  ---  edificio_agrícola
edificio industrial  ---  edificio_industrial
clima  ---  clima
industria de cosméticos  ---  industriar_cosmético
cría de ganado  ---  crío_ganar
aprove

vida animal  ---  vida_animal
control de calidad  ---  control_calidad
abono ecológico  ---  abonar_ecológico
hábitat urbano  ---  hábitat_urbano
sistema educativo  ---  sistema_educativo
equipamiento urbano  ---  equipamiento_urbano
arquitectura tradicional  ---  arquitectura_tradicional
composición de la población  ---  composición_población
daño por agua  ---  dañar_aguar
contaminante urbano  ---  contaminante_urbano
proceso tecnológico  ---  procesar_tecnológico
dimensión de la empresa  ---  dimensión_empresa
desarrollo biológico  ---  desarrollar_biológico
daño ambiental  ---  dañar_ambiental
agua urbana  ---  aguar_urbano
tratamiento del agua  ---  tratamiento_aguar
planificación de políticas  ---  planificación_político
planes de desarrollo  ---  plan_desarrollar
seguridad de instalaciones  ---  seguridad_instalación
protección del paisaje  ---  protección_paisaje
protección de paisaje  ---  protección_paisaje
transporte marítimo  ---  transportar_marítimo
industria extractiva  

tecnología energética  ---  tecnología_energético
tipo de suelo  ---  tipo_soler
biodiversidad  ---  biodiversidad
compensación financiera  ---  compensación_financiero
saneamiento urbano  ---  saneamiento_urbano
industria frigorífica  ---  industriar_frigorífico
reutilización del agua  ---  reutilización_aguar
fuerzas armadas  ---  fuerza_armar
plagas agrícolas  ---  plagar_agrícola
areas naturales  ---  areas_natural
vida de la empresa  ---  vida_empresa
condiciones sociales  ---  condicionar_social
industria azucarera  ---  industriar_azucarero
ingeniería marítima  ---  ingeniería_marítimo
derecho administrativo  ---  derecho_administrativo
modelo de desarrollo  ---  modelar_desarrollar
biología del suelo  ---  biología_soler
movimiento de población  ---  movimiento_población
comida rápida  ---  comer_rápido
arte  ---  arte
geografía económica  ---  geografía_económico
innovación  ---  innovación
estabilidad de suelo  ---  estabilidad_soler
agricultura de riego  ---  agricultura_reg

agua subterránea  ---  aguar_subterráneo
ciencias naturales  ---  ciencia_natural
ley (individual)  ---  ley_individual
ecosistemas urbanos  ---  ecosistema_urbano
gestión de existencias  ---  gestión_existencia
política de ayudas  ---  político_ayudar
ayuda al desarrollo  ---  ayudar_desarrollar
fondo financiero  ---  fondo_financiero
gestión de riesgo  ---  gestión_riesgo
monóxido de carbono  ---  monóxido_carbonar
intervención en la tierra  ---  intervención_tierra
animal para matadero  ---  animal_matadero
sacrificio de animales  ---  sacrificio_animal
distribución de la población  ---  distribución_población
enfermedades de animales  ---  enfermedad_animal
ocupación del suelo  ---  ocupación_soler
agua (geografía)  ---  aguar_geografía
asociación profesional  ---  asociación_profesional
movilidad espacial  ---  movilidad_espacial
migración (animal)  ---  migración_animal
control de erosión  ---  control_erosión
productos industriales  ---  producto_industrial
contaminación agrícol

...

polvillo radiactivo  ---  polvillo_radiactivo
fluidificación  ---  fluidificación
golf  ---  golf
amonificación  ---  amonificación
licencia de importación  ---  licenciar_importación
restricción comercial  ---  restricción_comercial
aparcamientos  ---  aparcamiento
monopolio  ---  monopolio
humo  ---  humo
pérdida  ---  pérdida
camping  ---  camping
hidrobiología  ---  hidrobiología
moluscos  ---  molusco
patente  ---  patentar
reintroducción  ---  reintroducción
permiso de importación de residuos  ---  permiso_importación_residuo
nitritos  ---  nitrito
gasificación  ---  gasificación
capacidad portante  ---  capacidad_portante
primates  ---  primate
descarga accidental de organismos  ---  descargar_accidental_organismo
accidente  ---  accidentar
contaminantes radiactivos  ---  contaminante_radiactivo
carga salada  ---  cargar_salar
esclusa  ---  esclusa
vertido de residuos  ---  verter_residuo
abejas  ---  abeja
lubricante  ---  lubricante
guerra  ---  guerra
mantenimiento de existen

ASEAN  ---  asear
sustancias radiactivas  ---  sustanciar_radiactivo
municipios  ---  municipio
libre cambio  ---  librar_cambiar
carbón  ---  carbón
volatilidad  ---  volatilidad
prueba aleatoria  ---  probar_aleatorio
puentes  ---  puente
vertido de residuos domésticos  ---  verter_residuo_doméstico
central de carbón  ---  central_carbón
percloroetileno  ---  percloroetileno
incineración  ---  incineración
prueba  ---  probar
grupo  ---  grupo
actínidos  ---  actínido
informe para el Ministro  ---  informar_ministro
carga de emisión  ---  cargar_emisión
carretera  ---  carretero
tapón salado  ---  tapón_salar
estanque fangoso  ---  estancar_fangoso
administración  ---  administración
residuo de incineración  ---  residuo_incineración
emulsificación  ---  emulsificación
incentivos contra la contaminación  ---  incentivo_contaminación
freón  ---  freón
ensayo  ---  ensayar
documento  ---  documentar
campo de vacaciones  ---  campar_vacación
campamento  ---  campamento
alergias  ---  al

cuero  ---  cuero
deber oficial  ---  deber_oficial
vandalismo  ---  vandalismo
=== EXCLUDE  neurotoxicity
laca  ---  laca
goma  ---  goma
cuerpo simple radiactivo  ---  cuerpo_simple_radiactivo
disolución  ---  disolución
sustancia mutagénica  ---  sustanciar_mutagénico
comparación  ---  comparación
miscibilidad  ---  miscibilidad
descentralización  ---  descentralización
cascada  ---  cascar
=== EXCLUDE  bradiseísmo
geodesia  ---  geodesia
ladrillos  ---  ladrillo
datos de prognosis  ---  dato_prognosis
motores de gasolina  ---  motor_gasolina
microorganismos  ---  microorganismo
soldadura  ---  soldadura
abono fuera de hora punta  ---  abonar_hora_punta
análisis de disposición a pagar  ---  análisis_disposición_a_pagar
pesca deportiva  ---  pescar_deportivo
reflexión  ---  reflexión
descuartizamiento  ---  descuartizamiento
sulfatos  ---  sulfato
pixel  ---  pixel
poder de purificación  ---  purificación
sanciones penales  ---  sancionar_penal
agente expedicionario  ---  agente_expe

combustibilidad  ---  combustibilidad
propulsor  ---  propulsor
biofísica  ---  biofísico
poda  ---  podar
información de referencia geográfica  ---  información_referenciar_geográfico
naúsea  ---  naúsea
habitantes  ---  habitante
ozonización  ---  ozonización
piel  ---  piel
calibrado  ---  calibrar
bóvidos  ---  bóvido
antagonismo  ---  antagonismo
nicho ahuecado por las olas  ---  nicho_ahuecar_ola
biología del aire  ---  biología_aire
boletín informativa  ---  boletín_informativo
dipteros  ---  dipteros
paleoclimatología  ---  paleoclimatología
aviso  ---  avisar
longitud  ---  longitud
citología  ---  citología
reflectometría  ---  reflectometría
extracción (de una muestra)  ---  extracción_mostrar
arboreto  ---  arboreto
alquitrán  ---  alquitrán
nivel sin efecto  ---  nivel_efecto
opinión  ---  opinión
alveolo  ---  alveolo
teratógenos  ---  teratógeno
bichos  ---  bicho
insonorización  ---  insonorización
basidiomiceto  ---  basidiomiceto
petróleo  ---  petróleo
normalización 

agroquímicos  ---  agroquímicos
agroindustria  ---  agroindustria
agrometeorología  ---  agrometeorología
desarrollo ecológico  ---  desarrollar_ecológico
etiquetado ecológico  ---  etiquetar_ecológico
ecotoxicidad  ---  ecotoxicidad
ecotoxicología  ---  ecotoxicología
electrosmog  ---  electrosmog
=== EXCLUDE  micetes
gallinacea  ---  gallinacea
instalación para producción de gas  ---  instalación_producción_gas
geotecnología  ---  geotecnología
cinturón verde  ---  cinturón_verde
=== EXCLUDE  haloforme
agricultura de riego  ---  agricultura_regar
infrasonido  ---  infrasonido
calefacción a distancia  ---  calefacción_a_distanciar
=== EXCLUDE  lisimetría
=== EXCLUDE  metainformation
=== EXCLUDE  microecosystem
microfiltración  ---  microfiltración
=== EXCLUDE  microcontaminante
mina  ---  minar
sobrexplotación  ---  sobrexplotación
=== EXCLUDE  superfertilización
=== EXCLUDE  nitrato de peroxiacetilo
fitomasa  ---  fitomasa
pinnípedos  ---  pinnípedo
plantígrado  ---  plantígrado
=== 

In [27]:
# adding all the words and topics to model

# load the lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()
# load the nolemma list
with open(nolemma_list_file_excluded) as fp:
    topics_nolemma = fp.readlines()    
    

#try if the topic-lemma words exist in the model     
new_vectors = []
new_words = []
for i, (topic, topic_nolemma) in enumerate(zip(topics, topics_nolemma)):
#for i,topic in enumerate(topics):
    topic_nolemma = re.sub('\W',' ',topic_nolemma)
    topic_nolemma = re.sub('\ +',' ',topic_nolemma)
    doc = nlp(topic_nolemma.strip().lower())
    nolemmas = []
    for token in doc:
        if token.is_stop:
            continue
        else:
            nolemmas.append(token.text) 
            
    topic = topic.strip()
    topic_words = topic.split('_')
    
    if len(topic_words) != len(nolemmas):
        print("!!!!! error - not equal length: ", nolemmas, topic_words)
        continue
    #if len(topic_words)<2:
    #    continue
    
    
    tmp_vectors = []
    
    for word, word_nolemma in zip(topic_words, nolemmas):
    #for word in topic_words:
        vector = get_vector(word)
        
        try:
            tmp_vectors.append(model[word])
        except: 
            print('\nNOT in vocabulary: ', word, topic)
            vector = get_vector(word)
            if vector != []:
                tmp_vectors.append(vector)
                if word not in new_words:
                    print('append ', word)
                    new_vectors.append(vector)
                    new_words.append(word)   
            else:
                vector = get_vector(word_nolemma)
                if vector != []:
                    tmp_vectors.append(vector)
                    if word not in new_words:
                        print('append ', word)
                        new_vectors.append(vector)
                        new_words.append(word)  
                else:
                    raise Warning(f"XXX {word} {topic} {topic_nolemma}")
                    print("XXX", word, topic, topic_nolemma) 
    #if word not in new_words:
    #    new_vectors.append(list(np.average(tmp_vectors, axis=0)))
    #    new_words.append(word)        
    if len(tmp_vectors) > 1:
        if topic not in new_words:
   
            print('append ', topic)
            new_vectors.append(list(np.average(tmp_vectors, axis=0)))
            new_words.append(topic)
        else:
            print('doubled TOPIC: ', topic)    
    else:
        print('--- only one word, not a new topic>', topic)

append  desarrollar_local
append  desarrollar_rural
append  gestión_recurso_aguar
append  nivel_enseñanza
--- only one word, not a new topic> enseñanza
--- only one word, not a new topic> ribera
append  derecho_comunitario
append  patrimonio_cultural
append  soler_agrícola
append  paisaje_agrícola
append  producción_agrícola
append  desarrollar_económico
append  cadena_alimentario
append  industriar_alimentación
--- only one word, not a new topic> nutritivo
append  calentamiento_tierra
append  efecto_invernadero
append  desarrollar_industrial
--- only one word, not a new topic> mercar
append  acceso_a_tierra
append  consolidación_soler
append  ordenación_territorio
append  arquitectura_paisaje
append  planificación_paisajístico
--- only one word, not a new topic> soler
append  planificación_tierra
append  ambiente_marino
append  conservación_naturaleza
append  protección_naturaleza
append  agricultura_biológico
append  planificación_físico
append  enseñanza_primario
append  industriar_

append  aguar_filtración
append  desigualdad_social
append  político_crediticio
append  gestión_costero
append  derecho_ambiental
append  transportar_acuático
append  conservación_aguar_dulce
append  aguar_residual_urbano
append  utilización_paisaje
--- only one word, not a new topic> natural
append  calidad_soler
append  industriar_mueble
append  estabilización_soler
append  arquitectura_rural
append  apoyar_económico
append  gestión_financiero
append  bien_consumir
append  tierra_anegar
append  político_cultural
append  industriar_hotelero
append  demandar_aguar
append  sector_económico
append  nivel_crítico
append  planificación_nacional
append  formación_soler
append  ecosistema_forestal
append  zona_costero
append  transportar_privar
doubled TOPIC:  explotación_agrícola
append  valoración_aguar
append  transportar_terrestre
append  sedimentar_marinar
append  transportar_público
append  terreno_árido
append  explotación_recurso
append  calidad_aguar
append  aguar_costero
append  ed

/home/dzon/kajo/kajo_semexp_core/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


append  densidad_población
append  erial_industrial
append  político_gubernamental
append  cooperación_desarrollar
--- only one word, not a new topic> sociedad
append  estructurar_industrial
append  estructurar_económico
append  contratación_administrativo
--- only one word, not a new topic> personar
append  contaminación_litoral
append  contaminación_aguar_dulce
append  granja_avícola
append  capacidad_productivo_suelo
append  mercar_ambientar
append  bombo_aguar
append  mercar_energía
--- only one word, not a new topic> conservación
append  nivel_aguar
append  construcción_agricola
--- only one word, not a new topic> empresa
append  obrar_público
append  seguridad_ambiental
append  mercar_monetario
append  actividad_económico
append  actividad_industrial
append  tendencia_población
append  necesidad_vivienda
append  político_producción
append  región_biogeográfico
append  sistema_comunicación
append  terreno_edificable
append  mercar_financiero
append  proceso_social
append  actuació

--- only one word, not a new topic> doméstico
append  verdura_ecológico
append  producción_alquitrán
append  plan_ambiental
append  gestión_pesquería
append  imponer_capital
append  ley_transportar
--- only one word, not a new topic> productividad
append  finanzas_comunitario
--- only one word, not a new topic> edad
append  político_energético
append  industriar_recuperación
append  extracción_aguar
append  alimento_básico
--- only one word, not a new topic> seguridad
append  cubrir_terrestre
append  contabilidad_nacional
append  contabilidad_ambiental
append  factor_climático
append  disolvente_orgánico
append  economía_negocio
append  gestión_montaña
append  reservar_natural
append  zona_residencial
append  estudiar_ambiental
append  tierra_marginal
append  subvención_ambiental
append  protección_plantar
append  hábitat_animal
append  derecho_acceso
append  político_presupuestario
append  químico_ambiental
append  reglamentar_construcción
append  industriar_papelero
append  ambientar

...

--- only one word, not a new topic> tubería
append  tipo_documentar
append  formar_gobernar
append  desechar_baterias
append  polvillo_radiactivo

NOT in vocabulary:  fluidificación fluidificación
append  fluidificación
--- only one word, not a new topic> fluidificación
--- only one word, not a new topic> golf

NOT in vocabulary:  amonificación amonificación
append  amonificación
--- only one word, not a new topic> amonificación
append  licenciar_importación
append  restricción_comercial
--- only one word, not a new topic> aparcamiento
--- only one word, not a new topic> monopolio
--- only one word, not a new topic> humo
--- only one word, not a new topic> pérdida
--- only one word, not a new topic> camping
--- only one word, not a new topic> hidrobiología
--- only one word, not a new topic> molusco
--- only one word, not a new topic> patentar
--- only one word, not a new topic> reintroducción
append  permiso_importación_residuo
--- only one word, not a new topic> nitrito
--- only one 

append  norma_contaminación
append  incinerador_residuo
append  humo_tabaco
append  hidrocarburo_clorado
append  riesgo_residual
--- only one word, not a new topic> subsidiar
append  fisión_nuclear
append  central_eléctrico_residuo_alimentar
--- only one word, not a new topic> asear
append  sustanciar_radiactivo
--- only one word, not a new topic> municipio
append  librar_cambiar
--- only one word, not a new topic> carbón
--- only one word, not a new topic> volatilidad
append  probar_aleatorio
--- only one word, not a new topic> puente
append  verter_residuo_doméstico
append  central_carbón

NOT in vocabulary:  percloroetileno percloroetileno
append  percloroetileno
--- only one word, not a new topic> percloroetileno
--- only one word, not a new topic> incineración
--- only one word, not a new topic> probar
--- only one word, not a new topic> grupo
--- only one word, not a new topic> actínido
append  informar_ministro
append  cargar_emisión
--- only one word, not a new topic> carretero

--- only one word, not a new topic> armar
--- only one word, not a new topic> político
append  organismo_objetivar
append  trinidad_principio
--- only one word, not a new topic> flor
--- only one word, not a new topic> geofísico
append  sustanciar_teratogénico
append  absorción_exposición
--- only one word, not a new topic> reposición
append  sustanciar_peligroso
append  corrección_geométrico
append  podar_propagación_vegetativo
--- only one word, not a new topic> aluminio
append  concentración_máximo_emisión
append  cama_fluidificar
append  osmosis_inverso
doubled TOPIC:  fotografiar_aéreo
--- only one word, not a new topic> decantación
--- only one word, not a new topic> mutagenicidad
--- only one word, not a new topic> pantano
doubled TOPIC:  radiación_ionizante
--- only one word, not a new topic> frigorífico
append  estudiar_escritorio
--- only one word, not a new topic> arañar
append  dosis_equivalente

NOT in vocabulary:  ignifugación agente_ignifugación
append  ignifugación
appe

--- only one word, not a new topic> necrosis
append  liberación_transformar_microorganismo
--- only one word, not a new topic> éter
--- only one word, not a new topic> ensenar
--- only one word, not a new topic> keroseno
append  co_incineración
--- only one word, not a new topic> delta
--- only one word, not a new topic> dosificación

NOT in vocabulary:  pentaclorofenol pentaclorofenol
append  pentaclorofenol
--- only one word, not a new topic> pentaclorofenol
doubled TOPIC:  separación_origen
--- only one word, not a new topic> escolar
--- only one word, not a new topic> eliminación
--- only one word, not a new topic> oncocercosis
--- only one word, not a new topic> emulsión
append  altura_excesivo_pilar_chimenea
--- only one word, not a new topic> ontogénesis
--- only one word, not a new topic> índice
append  agudeza_auditivo
append  rastro_caminata
--- only one word, not a new topic> germen
--- only one word, not a new topic> cucaracha
--- only one word, not a new topic> descargar
-

--- only one word, not a new topic> isómero
--- only one word, not a new topic> actinio
--- only one word, not a new topic> mutágeno

NOT in vocabulary:  olfatometría olfatometría
append  olfatometría
--- only one word, not a new topic> olfatometría
append  plasmar_germen
--- only one word, not a new topic> interpolación
--- only one word, not a new topic> color
--- only one word, not a new topic> silenciador
--- only one word, not a new topic> glosario
--- only one word, not a new topic> carcinógeno
append  cd_rom

NOT in vocabulary:  nitrosamina nitrosamina
append  nitrosamina
--- only one word, not a new topic> nitrosamina
--- only one word, not a new topic> trapo
--- only one word, not a new topic> desgrasar
--- only one word, not a new topic> iceberg
--- only one word, not a new topic> ablandamiento
--- only one word, not a new topic> gasohol

NOT in vocabulary:  reprocesado reprocesado
append  reprocesado
--- only one word, not a new topic> reprocesado
--- only one word, not a ne

In [28]:
# add topics to model
model.add(new_words,new_vectors)

In [29]:
# save the model with new topics
model.save_word2vec_format(excluded_model_name, binary=True)
#reload the model  otherwise cannot find similarities fo newly added words
model = KeyedVectors.load_word2vec_format(excluded_model_name, binary=True)

In [30]:
len(model.index2word)

169586

In [31]:
# try the model
model.similar_by_vector('desarrollar_rural')

[('rural', 0.9675341844558716),
 ('ambientar_rural', 0.9538182616233826),
 ('asentamiento_rural', 0.9367071986198425),
 ('arquitectura_rural', 0.9354815483093262),
 ('población_rural', 0.9129946231842041),
 ('turismo_rural', 0.8939685821533203),
 ('casar_rural', 0.8866465091705322),
 ('hábitat_rural', 0.8629573583602905),
 ('zona_rural', 0.8388579487800598),
 ('rurales', 0.7087579965591431)]

In [32]:
# find most similar to topics to terms in the new model

# load the new lemma list
with open(lemma_list_file_excluded) as fp:
    topics = fp.readlines()

for i, topic in enumerate(topics):
    topic = topic.strip()
    print('\n-----\n', topic)
    try:
        print(i, model.similar_by_vector(topic))
    except:
        print(topic, 'NOT in model')


-----
 desarrollar_local
0 [('local', 0.9540767669677734), ('gobernar_local', 0.9083348512649536), ('recreación_local', 0.8983229994773865), ('autoridad_local', 0.8824434280395508), ('hacienda_local', 0.8436385989189148), ('material_local_construcción', 0.8383170366287231), ('político_gobernar_local', 0.8371889591217041), ('servicio_local_pasajero', 0.8031210899353027), ('tráfico_local', 0.8029518127441406), ('utilización_recurso_local', 0.7881746292114258)]

-----
 desarrollar_rural
1 [('rural', 0.9675341844558716), ('ambientar_rural', 0.9538182616233826), ('asentamiento_rural', 0.9367071986198425), ('arquitectura_rural', 0.9354815483093262), ('población_rural', 0.9129946231842041), ('turismo_rural', 0.8939685821533203), ('casar_rural', 0.8866465091705322), ('hábitat_rural', 0.8629573583602905), ('zona_rural', 0.8388579487800598), ('rurales', 0.7087579965591431)]

-----
 gestión_recurso_aguar
2 [('gestión_aguar', 0.8660412430763245), ('gestión_recurso_natural', 0.844430685043335), ('

19 [('acceso_público_a_tierra', 0.9810850620269775), ('acceso_a_cultura', 0.9571611285209656), ('acceso_a_información', 0.9562047123908997), ('acceso_a_justicia', 0.9538819193840027), ('acceso_a_documento_administrativo', 0.932171106338501), ('derecho_a_información', 0.9218213558197021), ('derecho_a_compensación', 0.9115510582923889), ('volver_a_naturaleza', 0.9019895792007446), ('calefacción_a_distanciar', 0.9004944562911987), ('a', 0.8927559852600098)]

-----
 consolidación_soler
20 [('estabilización_soler', 0.9207170605659485), ('estructurar_soler', 0.8835693001747131), ('soler', 0.8775715827941895), ('estabilidad_soler', 0.864370584487915), ('formación_soler', 0.8642632961273193), ('compactación_soler', 0.8637744188308716), ('conservación_soler', 0.846098780632019), ('descontaminación_soler', 0.8323075771331787), ('degradación_soler', 0.8312869668006897), ('análisis_soler', 0.8033585548400879)]

-----
 ordenación_territorio
21 [('ordenación', 0.8541491031646729), ('ordenación_tierr

40 [('suelo', 0.890052318572998), ('mejoramiento_suelo', 0.868529200553894), ('mecánico_suelo', 0.8454196453094482), ('capacidad_productivo_suelo', 0.8116281032562256), ('erosión', 0.8088390827178955), ('erosión_hídrico', 0.7701901793479919), ('erosión_costero', 0.7591223120689392), ('suelos', 0.752665638923645), ('control_erosión', 0.7274637222290039), ('erosión_eólico', 0.6966308951377869)]

-----
 desarrollar_sostenible
41 [('sostenible', 0.8950570821762085), ('objetivo_desarrollar_sostenible', 0.8877561688423157), ('indicador_desarrollar_sostenible', 0.8534358739852905), ('desarrollar_ecológico', 0.7736191153526306), ('desarrollar_ambiental', 0.7241933941841125), ('gestión_forestal_sostenible', 0.7156459093093872), ('sustentable', 0.7118996381759644), ('sostenibles', 0.6721030473709106), ('desarrollar_urbano_planificar', 0.6719227433204651), ('concienciar_ecológico', 0.666711688041687)]

-----
 planificación_urbano
42 [('urbano', 0.9260739088058472), ('estructurar_urbano', 0.921021

63 [('vehículo_motor', 0.9530419707298279), ('contaminación_vehículo_motor', 0.9338043332099915), ('motor', 0.8958892822265625), ('emisión_vehículo_motor', 0.8651150465011597), ('motor_combustión', 0.8504975438117981), ('motor_gasolina', 0.840287983417511), ('motor_diesel', 0.8322800397872925), ('motor_tiempo', 0.8310153484344482), ('industriar_fabricación_vehículo', 0.8155444860458374), ('vehículo_eléctrico', 0.7601522207260132)]

-----
 planificación_y_gestión_rural
64 [('gestión_y_planificación_tierra', 0.9314471483230591), ('desarrollar_y_planificación_urbano', 0.9184867739677429), ('reglamentar_comunitario_gestión_ecológico_y_auditoría', 0.9083125591278076), ('institución_público_naturaleza_industrial_y_comercial', 0.887186586856842), ('intervención_naturaleza_y_paisaje', 0.8863797783851624), ('saneamiento_y_salud_público', 0.883886456489563), ('indicador_social_y_cultural', 0.8817375302314758), ('información_científico_y_técnico', 0.8693653345108032), ('y_movilización_crítico', 0

81 [('tierra', 0.8956557512283325), ('observación_tierra', 0.8797717094421387), ('planificación_tierra', 0.8795641660690308), ('conservación_tierra', 0.8742425441741943), ('formación_tierra', 0.8678160905838013), ('contaminación_tierra', 0.8618499040603638), ('intervención_tierra', 0.8526899218559265), ('rehabilitación_tierra', 0.8509249687194824), ('ordenación_tierra', 0.847041666507721), ('deposición_tierra', 0.8404003977775574)]

-----
 gestión_cantidad_aguar
82 [('gestión_aguar', 0.897429347038269), ('gestión_calidad_aguar', 0.8784095644950867), ('gestión_aguar_superficie', 0.8688641786575317), ('cantidad_aguar_residual', 0.8400710821151733), ('gestión_recurso_aguar', 0.820171594619751), ('gestión_aguar_municipal', 0.8103864789009094), ('calidad_aguar_residual', 0.7822624444961548), ('calidad_aguar', 0.772284984588623), ('directivo_calidad_aguar', 0.765363872051239), ('instalación_abastecimiento_aguar', 0.7623076438903809)]

-----
 político_conservación_naturaleza
83 [('conservació

...

5128 [('información_científico_y_técnico', 0.8573089838027954), ('gestión_y_planificación_tierra', 0.8569110035896301), ('indicador_social_y_cultural', 0.8472838401794434), ('reglamentar_comunitario_gestión_ecológico_y_auditoría', 0.8458107113838196), ('desarrollar_y_planificación_urbano', 0.844063401222229), ('institución_público_naturaleza_industrial_y_comercial', 0.8427612781524658), ('promoción_comerciar_y_industriar', 0.8422199487686157), ('acústico_ciencia_y_tecnología', 0.8419907093048096), ('saneamiento_y_salud_público', 0.8375753164291382), ('planificación_y_gestión_rural', 0.8307728171348572)]

-----
 conservación_situ
5129 [('situ', 0.9630184173583984), ('tratamiento_aguar_residual_in_situ', 0.711261510848999), ('in_situ', 0.6650643944740295), ('insitu', 0.661969780921936), ('conservación', 0.5930966734886169), ('análisis_in_vitro', 0.5883625745773315), ('conservación_naturaleza', 0.5505227446556091), ('análisis_in_vivir', 0.5385417938232422), ('programar_nacional_conservaci

5146 [('helar', 0.8235722780227661), ('capar', 0.7781763076782227), ('capar_soler', 0.7488812208175659), ('balance_masa_capar_helar', 0.7342252731323242), ('capar_freático', 0.6927292943000793), ('protección_capar_freático', 0.6814266443252563), ('pérdida_helar', 0.6692035794258118), ('capar_inversión', 0.6594135165214539), ('aguar_calentar', 0.6528447866439819), ('cascar', 0.6469825506210327)]

-----
 balance_masa_capar_helar
5147 [('transportar_masa', 0.7591457962989807), ('masa', 0.7358579635620117), ('capar_helar', 0.7342252731323242), ('separación_masa', 0.7230662107467651), ('masa_flotante_petróleo', 0.6946917772293091), ('infiltración_aguar_tierra', 0.6247706413269043), ('protección_capar_freático', 0.6049457788467407), ('pérdida_helar', 0.5991409420967102), ('cargar_aguar_residual', 0.5978936553001404), ('barrer_aguar_residual', 0.5976835489273071)]

-----
 degradación_soler
5148 [('contaminación_soler', 0.8736317157745361), ('descontaminación_soler', 0.8698559999465942), ('est

5166 [('ordenación_cuenca_hidrográfico', 0.9371969699859619), ('cuenca', 0.932110071182251), ('cuenca_sedimentación', 0.922298789024353), ('cuenca_sedimentario', 0.9142940044403076), ('cuenca_fluvial', 0.9024097919464111), ('desarrollar_cuenca_fluvial', 0.8879736065864563), ('cuenca_fluvial_internacional', 0.8844282031059265), ('cuenca_lacustre', 0.8801205158233643), ('cuenca_retardar', 0.8604297041893005), ('hidrográfico', 0.8101438879966736)]

-----
 ruido_tráfico
5167 [('control_ruido_tráfico', 0.9556088447570801), ('ruido', 0.882074236869812), ('ruido_intermitente', 0.8698871731758118), ('legislación_ruido', 0.8695425987243652), ('ruido_construcción', 0.8683841228485107), ('protección_ruido', 0.8640761375427246), ('ruido_aerodinámico', 0.8618647456169128), ('ruido_continuo', 0.8575559854507446), ('control_ruido', 0.8553800582885742), ('ruido_doméstico', 0.8542153835296631)]

-----
 objetivo_desarrollar_sostenible
5168 [('desarrollar_sostenible', 0.8877562284469604), ('indicador_des

5185 [('ecológico', 0.907802939414978), ('concienciar_ecológico', 0.8839070796966553), ('etiquetar_ecológico', 0.8525848388671875), ('etiquetar_ecológico_comunitario', 0.8500609397888184), ('evaluación_ecológico', 0.8142234683036804), ('civilización_ecológico', 0.8119843006134033), ('resiliencia_ecológico', 0.8012247085571289), ('adaptación_ecológico', 0.7998124361038208), ('capacidad_ecológico', 0.7974555492401123), ('comunidad_ecológico', 0.7896284461021423)]

-----
 etiquetar_ecológico
5186 [('etiquetar_ecológico_comunitario', 0.9345934391021729), ('ecológico', 0.8755007982254028), ('desarrollar_ecológico', 0.8525848388671875), ('concienciar_ecológico', 0.8479869961738586), ('comunidad_ecológico', 0.7677075862884521), ('evaluación_ecológico', 0.7639839053153992), ('resiliencia_ecológico', 0.7606148719787598), ('parámetro_ecológico', 0.7558555603027344), ('civilización_ecológico', 0.7533800005912781), ('contabilidad_ecológico', 0.7509902715682983)]

-----
 ecotoxicidad
5187 [('ecotox

5214 [('pozos', 0.6540272831916809), ('hoyo', 0.610500693321228), ('aljibe', 0.5872970819473267), ('barreno', 0.572985053062439), ('cantera_subterráneo', 0.568402111530304), ('brocal', 0.5650708675384521), ('manantial', 0.563159704208374), ('depósito_fosar_marino', 0.5595244765281677), ('drenaje_subsuelo', 0.5535014271736145), ('tanque_séptico', 0.5402883291244507)]

-----
 teneduría
5215 [('contabilidad', 0.6078987121582031), ('contable', 0.5315443277359009), ('gestión_contable', 0.5113635063171387), ('contaduría', 0.5107666254043579), ('llevanza', 0.5037964582443237), ('contabilidad_nacional', 0.47252973914146423), ('sistema_contabilidad', 0.47031715512275696), ('contabilización', 0.4699912667274475), ('guarnicionería', 0.45993882417678833), ('contabilidad_ambiental', 0.445814311504364)]
